In [1]:
import pandas as pd

In [2]:
from functions import read_data, clean_data, change_value, drop_columns_education, drop_columns_unemployment

In [3]:
url = "data/edat_lfs_9903__custom_12827842_linear.csv"
education =read_data(url)
pd.set_option('display.max_columns', None)
education.head()

,DATAFLOW,LAST UPDATE,freq,unit,sex,isced11,age,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2014,27.9,b
1,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2015,27.5,NaN
2,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2016,27.3,NaN
3,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2017,27.2,NaN
4,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2018,27.1,NaN


In [4]:
education = clean_data(education)
education.head()

,dataflow,last_update,freq,unit,sex,isced11,age,geo,time_period,obs_value,obs_flag
0,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2014,27.9,b
1,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2015,27.5,NaN
2,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2016,27.3,NaN
3,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2017,27.2,NaN
4,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,A:Annual,PC:Percentage,F:Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2018,27.1,NaN


In [ ]:
#print(education["sex"].unique())
#print()
#print(education["isced11"].unique())
#print()
#print(education["age"].unique())

In [5]:
education = change_value(education)
education.head()

,dataflow,last_update,freq,unit,sex,isced11,age,geo,time_period,obs_value,obs_flag
0,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,Anual,Percentage,Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2014,27.9,b
1,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,Anual,Percentage,Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2015,27.5,NaN
2,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,Anual,Percentage,Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2016,27.3,NaN
3,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,Anual,Percentage,Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2017,27.2,NaN
4,ESTAT:EDAT_LFS_9903(1.0),13/06/24 23:00:00,Anual,Percentage,Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2018,27.1,NaN


In [10]:
url2 = r"data/yth_empl_090__custom_12832588_linear.csv"
unemployment = read_data(url2)
unemployment.head()

,DATAFLOW,LAST UPDATE,freq,sex,age,unit,isced11,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2014,15.7,b
1,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2015,15.3,NaN
2,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2016,17.1,NaN
3,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2017,16.8,NaN
4,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2018,13.4,NaN


In [11]:
unemployment = clean_data(unemployment)
unemployment.head()

,dataflow,last_update,freq,sex,age,unit,isced11,geo,time_period,obs_value,obs_flag
0,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2014,15.7,b
1,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2015,15.3,NaN
2,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2016,17.1,NaN
3,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2017,16.8,NaN
4,ESTAT:YTH_EMPL_090(1.0),13/06/24 23:00:00,A,F,Y15-29,PC,ED0-2,AT,2018,13.4,NaN


In [9]:
education.shape

(6534, 11)

In [10]:
unemployment.shape

(13734, 11)

In [7]:
drop_columns_education(education)
education

,dataflow,sex,isced11,age,geo,time_period,obs_value
0,ESTAT:EDAT_LFS_9903(1.0),Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2014,27.9
1,ESTAT:EDAT_LFS_9903(1.0),Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2015,27.5
2,ESTAT:EDAT_LFS_9903(1.0),Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2016,27.3
3,ESTAT:EDAT_LFS_9903(1.0),Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2017,27.2
4,ESTAT:EDAT_LFS_9903(1.0),Females,"ED0-2:Less than primary, primary and lower sec...",Y15-29:From 15 to 29 years,AT:Austria,2018,27.1
...,...,...,...,...,...,...,...
6529,ESTAT:EDAT_LFS_9903(1.0),Total,ED5-8:Tertiary education (levels 5-8),Y15-29:From 15 to 29 years,UK:United Kingdom,2015,28.9
6530,ESTAT:EDAT_LFS_9903(1.0),Total,ED5-8:Tertiary education (levels 5-8),Y15-29:From 15 to 29 years,UK:United Kingdom,2016,29.3
6531,ESTAT:EDAT_LFS_9903(1.0),Total,ED5-8:Tertiary education (levels 5-8),Y15-29:From 15 to 29 years,UK:United Kingdom,2017,29.5
6532,ESTAT:EDAT_LFS_9903(1.0),Total,ED5-8:Tertiary education (levels 5-8),Y15-29:From 15 to 29 years,UK:United Kingdom,2018,30.4


In [7]:
drop_columns_unemployment(unemployment)
unemployment

,sex,age,isced11,geo,TIME_PERIOD,OBS_VALUE
0,F,Y15-29,ED0-2,AT,2014,15.7
1,F,Y15-29,ED0-2,AT,2015,15.3
2,F,Y15-29,ED0-2,AT,2016,17.1
3,F,Y15-29,ED0-2,AT,2017,16.8
4,F,Y15-29,ED0-2,AT,2018,13.4
...,...,...,...,...,...,...
13729,T,Y15-29,TOTAL,UK,2015,882.1
13730,T,Y15-29,TOTAL,UK,2016,769.5
13731,T,Y15-29,TOTAL,UK,2017,688.1
13732,T,Y15-29,TOTAL,UK,2018,649.3
